In [1]:
import ee
import numpy as np
import concurrent
import os
from utils import *

In [3]:
# Initialize the Earth Engine module
ee.Initialize()

# Region of interest (bounding box of region containing glacier outlines)
# roi = [86.6, 27.7, 87.0, 28.0]
bounds = ee.FeatureCollection('projects/lia-moraine-id/assets/lia_extents_endofws').geometry().bounds().getInfo()
coords = np.array(bounds['coordinates'])
x = coords[0,:,0]
y = coords[0,:,1]
roi = [x.min(), y.min(), x.max(), y.max()]

# Landsat 8 surface reflectance composite
dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(ee.Geometry.Rectangle(roi))
dataset = dataset.map(lambda image: set_resampling_method_on_collection(image, 'bilinear'))
composite = create_composite_landsat8_sr(dataset, False)
blue = composite.select("SR_B2")
green = composite.select("SR_B3")
red = composite.select("SR_B4")
near_infrared = composite.select("SR_B5")

# NASA DEM
dataset = ee.Image('NASA/NASADEM_HGT/001').resample('bilinear')
elevation = dataset.select('elevation')

# Moraines mask
moraines_mask = create_moraines_mask('projects/lia-moraine-id/assets/lia_extents_endofws', 'label>0', True)

# List of image sources to fetch patches from (will add more to this list later)
image_sources = [
    {'image': blue, 'name': 'landsat8_blue'},
    {'image': green, 'name': 'landsat8_green'},
    {'image': red, 'name': 'landsat8_red'},
    {'image': near_infrared, 'name': 'landsat8_nir'},
    {'image': elevation, 'name': 'nasadem'},
    {'image': moraines_mask, 'name': 'moraines_mask'}
]

In [5]:
# Output file format
file_format = 'GEO_TIFF'

# Specify output spatial resolution of 30 m, get scale in units of degrees
scale = 30
projection = ee.Projection('EPSG:4326').atScale(scale).getInfo()
tform = projection['transform']
scale_x = tform[0]
scale_y = -tform[4]

# Patch stride and size (units of degrees)
step_x = 0.025
step_y = 0.025
width = 0.05
height = 0.05

# Initialize
patches = []
id = 0

# Loop through image patch windows
for x in np.arange(roi[0], roi[2], step_x):
    for y in np.arange(roi[1], roi[3], step_y):
        for source in image_sources:

            # Data for fetch_pixels()
            patches.append({
                'image': source['image'],
                'file_format': file_format,
                'width': width,
                'height': height,
                'translate_x': x,
                'translate_y': y,
                'scale_x': scale_x,
                'scale_y': scale_y,
                'crs': projection['crs'],
                'name': source['name'],
                'id': id
            })

        # Increment patch id
        id += 1

In [6]:
# Directories for saving image patches
parent_dir = 'image_patches/'
for source in image_sources:
    path = parent_dir + source['name']
    if not os.path.isdir(path):
        os.makedirs(path)  

# Concurrency library to download multiple image patches in parallel
# Use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:

    # Start the fetch operations
    future_to_pixels = {executor.submit(fetch_pixels, patch): patch for patch in patches}
    for future in concurrent.futures.as_completed(future_to_pixels):
        pixels = future_to_pixels[future]
        try:
            
            # Write geotiff
            data = future.result()
            with open(parent_dir + str(data['name']) + '/' + str(data['name']) + '_' + str(data['id']) + '.tif', 'wb') as f: 
                f.write(data['pixels'])

        except Exception as e:
            print(e)
            pass